In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import os
import sys

In [2]:
data_dir = './data'

student_log_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.startswith('student_log')]
training_label_path = os.path.join(data_dir, 'training_label.csv')
validation_test_label = os.path.join(data_dir, 'validation_test_label.csv')

dfs = []
for path in student_log_paths:
    temp = pd.read_csv(path)
    dfs.append(temp)
student_df = pd.concat(dfs)

training_label_df = pd.read_csv(training_label_path)
validation_test_label_df = pd.read_csv(validation_test_label)

C:\Users\YEUNG\Anaconda3\envs\dlenv\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print("student_df.shape:", student_df.shape) 
print("training_label_df.shape:", training_label_df.shape)
print("validation_test_label_df.shape:", validation_test_label_df.shape)

student_df.shape: (942816, 77)
training_label_df.shape: (514, 5)
validation_test_label_df.shape: (172, 4)


In [4]:
len(set(training_label_df.ITEST_id).intersection(set(validation_test_label_df.ITEST_id)))

48

In [5]:
student_df.head()

,AveCarelessness,AveCorrect,AveKnow,AveResBored,AveResConf,AveResEngcon,AveResFrust,AveResGaming,AveResOfftask,ITEST_id,...,timeOver80,timeSinceSkill,timeTaken,totalFrAttempted,totalFrPastWrongCount,totalFrPercentPastWrong,totalFrSkillOpportunities,totalFrSkillOpportunitiesByScaffolding,totalFrTimeOnSkill,totalTimeByPercentCorrectForskill
0,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,49.0,0,0,0.0,0,0.0,0.0,0.000000
1,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,4.0,1,0,0.0,1,0.0,49.0,106.000000
2,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,6.0,2,0,0.0,0,0.0,0.0,0.000000
3,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,18.0,3,1,0.0,1,0.0,0.0,0.000000
4,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,2.0,3,1,1.0,1,1.0,6.0,77.999999


In [6]:
student_df.columns

Index(['AveCarelessness', 'AveCorrect', 'AveKnow', 'AveResBored', 'AveResConf',
       'AveResEngcon', 'AveResFrust', 'AveResGaming', 'AveResOfftask',
       'ITEST_id', 'Ln', 'Ln-1', 'NumActions', 'Prev5count', 'RES_BORED',
       'RES_CONCENTRATING', 'RES_CONFUSED', 'RES_FRUSTRATED', 'RES_GAMING',
       'RES_OFFTASK', 'SY ASSISTments Usage', 'actionId', 'assignmentId',
       'assistmentId', 'attemptCount', 'bottomHint', 'confidence(BORED)',
       'confidence(CONCENTRATING)', 'confidence(CONFUSED)',
       'confidence(FRUSTRATED)', 'confidence(GAMING)', 'confidence(OFF TASK)',
       'consecutiveErrorsInRow', 'correct', 'endTime',
       'endsWithAutoScaffolding', 'endsWithScaffolding', 'frIsHelpRequest',
       'frIsHelpRequestScaffolding', 'frPast5HelpRequest', 'frPast5WrongCount',
       'frPast8HelpRequest', 'frPast8WrongCount', 'frTimeTakenOnScaffolding',
       'frTotalSkillOpportunitiesScaffolding', 'frWorkingInSchool',
       'helpAccessUnder2Sec', 'hint', 'hintCount', 'hin

In [7]:
len(student_df.ITEST_id.unique())

1709

In [8]:
skill = student_df.skill.unique()
skill_id = range(len(skill))
skill_id_map = dict(zip(skill, skill_id))

In [9]:
len(skill)

102

In [10]:
dkt_df = student_df[['ITEST_id', 'correct', 'skill', 'actionId']]
dkt_df['skill_id'] = dkt_df['skill'].apply(lambda x: skill_id_map[x])
dkt_df['user_id'] = dkt_df['ITEST_id']
dkt_df['time_idx'] = dkt_df['actionId']
sorted_data = dkt_df.sort_values('time_idx')
sorted_data.head()

C:\Users\YEUNG\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\YEUNG\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\YEUNG\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,ITEST_id,correct,skill,actionId,skill_id,user_id,time_idx
0,8,0,properties-of-geometric-figures,9950,0,8,9950
1,8,1,properties-of-geometric-figures,9951,0,8,9951
2,8,0,sum-of-interior-angles-more-than-3-sides,9952,1,8,9952
3,8,0,sum-of-interior-angles-more-than-3-sides,9953,1,8,9953
4,8,1,sum-of-interior-angles-more-than-3-sides,9954,1,8,9954


# Prepare data to train DKT model (already done)

In [11]:
REQUIRE_COLS = ['time_idx', 'user_id', 'skill_id', 'correct']
user_ids = dkt_df.user_id.unique()
tuples = []
for id in user_ids:
    df = sorted_data[sorted_data.user_id == id]
    df = df[REQUIRE_COLS]
    problems = [int(pid) for pid in df.skill_id]
    corrects = [int(corr) for corr in df.correct]
    num_problems = len(problems)
#     print (num_problems)
#     print (problems)
#     print (corrects)
#     print ("============")
    result = (num_problems, problems, corrects)
    tuples.append(result)

In [12]:
train, test = train_test_split(tuples, test_size=0.2)

In [13]:
if False:
    with open('./data/assistment_challenge/assistment_challenge_train.csv', 'w') as f:
        writer = csv.writer(f, 
                            delimiter=',', 
                            quotechar="'", 
                            quoting=csv.QUOTE_MINIMAL,
                            lineterminator='\n')
        for tup in train:
            writer.writerow([tup[0]])
            writer.writerow(tup[1])
            writer.writerow([int(i) for i in tup[2]])
            
    with open('./data/assistment_challenge/assistment_challenge_test.csv', 'w') as f:
        writer = csv.writer(f, 
                            delimiter=',', 
                            quotechar="'", 
                            quoting=csv.QUOTE_MINIMAL,
                            lineterminator='\n')
        for tup in test:
            writer.writerow([tup[0]])
            writer.writerow(tup[1])
            writer.writerow([int(i) for i in tup[2]])

In [14]:
student_specific_columns = ["AveKnow",
                            "AveCarelessness",
                            "AveCorrect",
                            "NumActions",
                            "AveResBored",
                            "AveResEngcon",
                            "AveResConf",
                            "AveResFrust",
                            "AveResOfftask",
                            "AveResGaming"]

In [15]:
required_cols = ['ITEST_id'] + student_specific_columns
student_specific_df = student_df[required_cols].drop_duplicates()

In [16]:
student_specific_df.head()

,ITEST_id,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,AveResConf,AveResFrust,AveResOfftask,AveResGaming
0,8,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,0.156503,0.196561
1056,35,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,0.153147,0.236800
2049,39,0.281693,0.152227,0.454545,418,0.274700,0.628075,0.116390,0.143827,0.221926,0.019817
2467,64,0.157938,0.098357,0.334038,1419,0.198394,0.670062,0.085895,0.132328,0.115773,0.303426
3886,77,0.191948,0.094195,0.413249,317,0.261455,0.638636,0.063971,0.129858,0.234331,0.045096


In [17]:
training_label_df.head()

,ITEST_id,SchoolId,AveCorrect,MCAS,isSTEM
0,9,2,0.438492,32,1
1,27,1,0.348837,21,0
2,33,2,0.686391,52,0
3,35,2,0.379658,34,0
4,37,3,0.305785,-999,0


In [18]:
# combined_df = pd.merge(left=training_label_df, right=student_specific_df, how='left', left_on='ITEST_id', right_on='ITEST_id')
combined_df_train = training_label_df.merge(student_specific_df, left_on='ITEST_id', right_on='ITEST_id', how='left')
combined_df_train['AveCorrect'] = combined_df_train['AveCorrect_x']
del combined_df_train['AveCorrect_x']
del combined_df_train['AveCorrect_y']
X = combined_df_train[student_specific_columns].values
y = combined_df_train['isSTEM'].values

In [19]:
combined_df_test = pd.merge(left=validation_test_label_df, right=student_specific_df, how='left')
X_target = combined_df_test[student_specific_columns].values

In [20]:
X.shape

(514, 10)

In [21]:
combined_df_train.head()

,ITEST_id,SchoolId,MCAS,isSTEM,AveKnow,AveCarelessness,NumActions,AveResBored,AveResEngcon,AveResConf,AveResFrust,AveResOfftask,AveResGaming,AveCorrect
0,9,2,32,1,0.185138,0.099734,504,0.277149,0.644744,0.098078,0.162771,0.213378,0.005554,0.438492
1,27,1,21,0,0.142031,0.069297,129,0.330226,0.551367,0.122658,0.095420,0.348090,0.036900,0.348837
2,33,2,52,0,0.459813,0.202787,169,0.260426,0.650769,0.059971,0.061834,0.296286,0.010954,0.686391
3,35,2,34,0,0.255164,0.158848,993,0.222796,0.650079,0.069987,0.164347,0.153147,0.236800,0.379658
4,37,3,-999,0,0.071909,0.046183,121,0.326384,0.653445,0.125325,0.094714,0.451467,0.016241,0.305785


# Import model of DKT
If you want to change the model with different regularization parameters
Simply change
```python
network_config['lambda_o']
network_config['lambda_w1']
network_config['lambda_w2']
```
to the desired value

---
To use the original DKT model without any regularization:
```python
network_config['lambda_o'] = 0.0
network_config['lambda_w1'] = 0.0
network_config['lambda_w2'] = 0.0
```

To use the original DKT model with regularization:
```python
network_config['lambda_o'] = 0.1
network_config['lambda_w1'] = 0.3
network_config['lambda_w2'] = 3.0
```

In [22]:
import os
import tensorflow as tf
import numpy as np

from utils import DKT
from load_data import DKTData

In [23]:
train_path = './data/assistment_challenge/assistment_challenge_train.csv'
test_path = './data/assistment_challenge/assistment_challenge_test.csv'
save_dir_prefix = './dkt_models/assistment_challenge/'

In [27]:
rnn_cells = {
    "LSTM": tf.contrib.rnn.LSTMCell,
    "GRU": tf.contrib.rnn.GRUCell,
    "BasicRNN": tf.contrib.rnn.BasicRNNCell,
    "LayerNormBasicLSTM": tf.contrib.rnn.LayerNormBasicLSTMCell,
}

# train_path = os.path.join('./data/', 'skill_id_train.csv')
# test_path = os.path.join('./data/', 'skill_id_test.csv')

network_config = {}
network_config['batch_size'] = 32
network_config['hidden_layer_structure'] = [200]
network_config['learning_rate'] = 0.01
network_config['keep_prob'] = 0.333
network_config['rnn_cell'] = rnn_cells["LSTM"]

network_config['lambda_o'] = 0.0
network_config['lambda_w1'] = 0.0
network_config['lambda_w2'] = 0.0

num_runs = 1
num_epochs = 1
batch_size = 32
keep_prob = 0.333

In [28]:
tf.reset_default_graph()

In [29]:
sess = tf.Session()

data = DKTData(train_path, test_path, batch_size=batch_size)
data_train = data.train
data_test = data.test
num_problems = data.num_problems

dkt = DKT(sess, data_train, data_test, num_problems, network_config,
          num_runs=num_runs, num_epochs=num_epochs,
          save_dir_prefix=save_dir_prefix,
          keep_prob=keep_prob, logging=False, save=False)

# load the model
dkt.model.build_graph()
dkt.load_model()

Reading ./data/assistment_challenge/assistment_challenge_train.csv
4101 lines was read
max_num_problems_answered: 3057
num_problems: 102
The number of students is 1366
Finish reading data.
Reading ./data/assistment_challenge/assistment_challenge_test.csv
1026 lines was read
max_num_problems_answered: 2107
num_problems: 101
The number of students is 342
Finish reading data.
ckpt_save_dir:  ./dkt_models/assistment_challenge/checkpoints\n200.lo0.0.lw10.0.lw20.0/
log_save_dir:  ./dkt_models/assistment_challenge/logs\n200.lo0.0.lw10.0.lw20.0/
Network Configuration:
batch_size: 32
hidden_layer_structure: [200]
learning_rate: 0.01
keep_prob: 0.333
rnn_cell: <class 'tensorflow.python.ops.rnn_cell_impl.LSTMCell'>
lambda_o: 0.0
lambda_w1: 0.0
lambda_w2: 0.0
Num of problems: 102
Num of run: 1
Max num of run: 1
Keep Prob: 0.333
Creating placeholder...
Creating Loss...
Creating Loss...
Create optimizer...
INFO:tensorflow:Restoring parameters from ./dkt_models/assistment_challenge/checkpoints\n200.l

In [ ]:
# Evaluate the model if you want.
_evaluation = True
if _evaluation:
    auc_n, auc_c, entropy_loss = dkt.evaluate()
    print('auc_n:', auc_n)
    print('auc_c:', auc_c)
    
    w1, w2 = dkt.waviness()
    print('w1:', w1)
    print('w2:', w2)
    
    m1, m2 = dkt.consistency()
    print('m1:', m1)
    print('m2:', m2)

# Obtain student's DKT feature

In [ ]:
import os
os.path.exists('data/students_knowledge_states.csv')

In [ ]:
# making the feature vector of student
import os
if os.path.exists('students_knowledge_states.csv'):
    students_dk_df = pd.read_csv('students_knowledge_states.csv')
else:
    REQUIRE_COLS = ['time_idx', 'user_id', 'skill_id', 'correct']
    user_ids = list(set(list(combined_df_train.ITEST_id.unique()) + list(combined_df_test.ITEST_id.unique())))
    kt_columns_name = ['kt_{}'.format(i) for i in range(102)]
    columns_name = kt_columns_name + ['ITEST_id']
    knowledge_states = []

    for i, id in enumerate(user_ids):
        if i % 10 == 0:
            print(i, id, end='\r')

        df = sorted_data[sorted_data.user_id == id]
        df = df[REQUIRE_COLS]
        problems = [int(pid) for pid in df.skill_id]
        corrects = [int(corr) for corr in df.correct]
        num_problems = len(problems)

        # the student knowledge state in the last output
        skt = dkt.get_output_layer([problems], [corrects])[0][-1]

        knowledge_state = dict(zip(kt_columns_name, skt))
        knowledge_state['ITEST_id'] = id
        knowledge_states.append(knowledge_state)

    students_dk_df = pd.DataFrame(knowledge_states)    
    students_dk_df['ITEST_id'] = students_dk_df['ITEST_id'].apply(int)
    
    students_dk_df.to_csv('students_knowledge_states.csv', header='True', index=False)

In [ ]:
combined_df = pd.merge(left=student_specific_df, right=students_dk_df, how='left')
train_df = training_label_df.merge(combined_df, left_on='ITEST_id', right_on='ITEST_id', how='left')
test_df = validation_test_label_df.merge(combined_df, left_on='ITEST_id', right_on='ITEST_id', how='left')
train_df['AveCorrect'] = train_df['AveCorrect_x']
del train_df['AveCorrect_x']
del train_df['AveCorrect_y']
test_df['AveCorrect'] = test_df['AveCorrect_x']
del test_df['AveCorrect_x']
del test_df['AveCorrect_y']

In [ ]:
features_cols = list(train_df.columns.drop(['ITEST_id', 'isSTEM']))
X_train_all = train_df[features_cols].values
y_train_all = train_df['isSTEM'].values
X_test_all = test_df[features_cols].values

# Training the static machine learning model

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import auc, roc_curve, mean_squared_error, make_scorer

In [ ]:
# prepare the train data and test data
cv = StratifiedShuffleSplit(n_splits=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.2)

In [ ]:
lr_param_grid = {
    'penalty': ['l1', 'l2']
}

In [ ]:
svm_param_grid = {
    'C': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0, 100.0], 
    'gamma': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0, 100.0], 
    'kernel': ['rbf']
}

In [ ]:
gbdt_param_grid = {
    'n_estimators': [200, 250, 300, 350, 400, 450, 500, 550], 
    'max_depth': [2, 3, 4, 5],
    'min_samples_leaf': [2, 3, 4, 5, 6, 7]
}

In [ ]:
import itertools
mlp_layer_search_list = list(itertools.product([16, 64, 128, 256]))
mlp_layer_search_list += list(itertools.product([16, 64, 128, 256], [16, 64, 128, 256]))
mlp_layer_search_list += list(itertools.product([16, 64, 128, 256], [16, 64, 128, 256], [16, 64, 128, 256]))
mlp_param_grid = {
    'hidden_layer_sizes': mlp_layer_search_list,
    'early_stopping': [True]
}

In [ ]:
def score_func_auc_rmse(y_true, y_pred):
    # auc
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
    auc_ = auc(fpr, tpr)
    
    # rmse
    rmse_ = np.sqrt(mean_squared_error(y_true, y_pred))
    
    return auc_ + (1 - rmse_)

admc_scorer = make_scorer(score_func_auc_rmse, greater_is_better=True)

In [ ]:
lr_clf = GridSearchCV(estimator=LogisticRegression(),
                  param_grid=lr_param_grid,
                  scoring=admc_scorer,
                  refit = True,
                  cv=cv,
                  verbose=1,
                  # n_jobs=2,
                  return_train_score=True)
lr_clf.fit(X_train, y_train)

In [ ]:
svm_clf = GridSearchCV(estimator=SVC(),
                  param_grid=svm_param_grid,
                  scoring=admc_scorer,
                  refit = True,
                  cv=cv,
                  verbose=1,
                  # n_jobs=2,
                  return_train_score=True)
svm_clf.fit(X_train, y_train)

In [ ]:
gbdt_clf = GridSearchCV(estimator=GradientBoostingClassifier(),
                  param_grid=gbdt_param_grid,
                  scoring=admc_scorer,
                  refit = True,
                  cv=cv,
                  verbose=1,
                  # n_jobs=2,
                  return_train_score=True)
gbdt_clf.fit(X_train, y_train)

In [ ]:
mlp_clf = GridSearchCV(estimator=MLPClassifier(),
                  param_grid=mlp_param_grid,
                  scoring=admc_scorer,
                  refit = True,
                  cv=cv,
                  verbose=1,
                  # n_jobs=2,
                  return_train_score=True)
mlp_clf.fit(X_train, y_train)

In [ ]:
def print_cv_result(clf, verbose=False):
    print("Best parameters set found on development set:")
    print(clf.best_params_)
    print("with auc + rmse:", max(clf.cv_results_['mean_test_score']))
    print("test result is")
    print(clf.score(X_test, y_test))
    if verbose:
        print("Grid scores on development set:")
        print('='*50)
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()
        
    y_pred = clf.predict(X_test)
    
    # find the auc
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    auc_ = auc(fpr, tpr)
    rmse_ = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print("test AUC:", auc_)
    print("test RMSE:", rmse_)
    print("AUC + (1-RMSE):", auc_ + (1 - rmse_))

In [ ]:
print_cv_result(lr_clf)

In [ ]:
print_cv_result(svm_clf)

In [ ]:
print_cv_result(gbdt_clf)

In [ ]:
print_cv_result(mlp_clf)

In [ ]:
print(svm_clf.score(X_test, y_test))

In [ ]:
print(gbdt_clf.score(X_test, y_test))

In [ ]:
print(mlp_clf.score(X_test, y_test))

In [ ]:
clf = gbdt_clf

In [ ]:
y_test_all = clf.predict_proba(X_test_all)
# prediction submit result
result = ','.join(["{:.5f}".format(i[1]) for i in y_test_all])
print(result)